# Predictive Models Based on Radiomics Features

This notebook contains the code for extracting radiomics features from the ultrasound data. Random forest classifiers are trained to predict the specified target variable from the extracted features.

In [ ]:
# Imports
import json
import logging
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import radiomics
import SimpleITK as sitk
from radiomics import featureextractor
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_classif
from sklearn.metrics import (accuracy_score, auc, balanced_accuracy_score,
                             confusion_matrix, f1_score, make_scorer,
                             precision_recall_curve, roc_auc_score, det_curve)
from sklearn.model_selection import GridSearchCV, StratifiedGroupKFold
from sklearn.pipeline import Pipeline
from tqdm import tqdm

sys.path.insert(0, '../')

In [ ]:
# Important constants
# TODO: fill in relevant directories

# Flag indicating whether to run k-fold CV: if False, models are evaluated using train-test split
VALIDATE = False
# Number of the folds for the CV
K_FOLDS = 5
SEED = 0
# Directory for logging
LOG_DIR = '...'
# Directory with the preprocessed ultrasound images
IMAGE_DIR = '...'
# Directory with data dictionaries
DICT_DIR = '...'
# Configuration file with the parameters for extracting radiomics features
RADIOMICS_CONFIG = './radiomics_params.yaml'
# Target variable: 'diagnosis', 'treatment' or 'complications'
TARGET_LABEL = 'diagnosis'

In [ ]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename=os.path.join(LOG_DIR, 'testLog.txt'), mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

## Data loading 

In [ ]:
# Load training images and create masks
# NOTE: the entire US image is used to ectract features, i.e. no ROI is specified
image_dir = os.path.join(IMAGE_DIR, 'constant_padding/deepfilled_cropped_train')

with open(os.path.join(DICT_DIR, TARGET_LABEL, 'imputed/final/app_data_train')) as f:
    gen_labels = json.load(f)

images = []
labels = []
groups = []
masks = []

for i in range(len(gen_labels)):
    img_code = list(gen_labels)[i]
    label = list(gen_labels.values())[i][1]
    file_names = list(gen_labels.values())[i][0]
    for file_name in file_names:
        images.append(sitk.ReadImage(os.path.join(image_dir, file_name), sitk.sitkInt32))
        labels.append([file_name, label])
    groups.extend(np.repeat(int(img_code), len(file_names)))
# Create a full mask
# GetImageFromArray reverses axes!
for i in range(len(images)):
    mask_arr = np.ones(images[i].GetSize()[::-1])
    mask_arr[0, 0] = 0 # pyradiomics wants mask to be segmented -> dummy segmentation
    mask = sitk.GetImageFromArray(mask_arr)
    mask.CopyInformation(images[i])  # copy geometric info
    masks.append(mask)

In [ ]:
# Load test images and create masks
if not VALIDATE:
    image_dir_test = os.path.join(IMAGE_DIR, 'constant_padding/deepfilled_cropped_test')    
    with open(os.path.join(DICT_DIR, TARGET_LABEL, 'imputed/final/app_data_test')) as f:
        gen_labels_test = json.load(f)

    images_test = []
    labels_test = []
    groups_test = []
    masks_test = []
    
    for i in range(len(gen_labels_test)):
        img_code = list(gen_labels_test)[i]
        label = list(gen_labels_test.values())[i][1]
        file_names = list(gen_labels_test.values())[i][0]
        for file_name in file_names:
            images_test.append(sitk.ReadImage(os.path.join(image_dir_test, file_name), sitk.sitkInt32))
            labels_test.append([file_name, label])
        groups_test.extend(np.repeat(int(img_code), len(file_names)))
    # Create a full mask
    # GetImageFromArray reverses axes!
    for i in range(len(images_test)):
        mask_arr = np.ones(images_test[i].GetSize()[::-1])
        mask_arr[0, 0] = 0 # pyradiomics wants mask to be segmented -> dummy segmentation
        mask = sitk.GetImageFromArray(mask_arr)
        mask.CopyInformation(images_test[i])  # copy geometric info
        masks_test.append(mask)

In [ ]:
%matplotlib inline

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(sitk.GetArrayFromImage(images[10]), cmap='gray')
plt.title('Abdomen')
plt.subplot(1,2,2)
plt.imshow(sitk.GetArrayFromImage(masks[10]))        
plt.title('Mask')
plt.axis('off')
plt.show()

## Radiomic features extraction

In [ ]:
# Use parameters from the configuration file
extractor = featureextractor.RadiomicsFeatureExtractor(RADIOMICS_CONFIG)
print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

In [ ]:
# Build the design matrix
feature_names = []
X = []
with tqdm(total=len(images)) as pbar:
    for i in range(len(images)):
        i_features = []
        featureVector = extractor.execute(images[i], masks[i])    
        for key, value in featureVector.items():
            if i == 0 and 'diagnostics' not in key:
                feature_names.append(key)
            if 'diagnostics' not in key:        
                i_features.append(float(value))
        X.append(i_features)
        pbar.update(1)

# Average per-view features for every subject
X = np.array(X)
for j in np.unique(groups):
    j_inds = np.argwhere(groups == j)[:, 0]
    X[j_inds] = np.tile(np.mean(X[j_inds], 0), (len(j_inds), 1))

X_df = pd.DataFrame(X, columns=feature_names)
labels_df = pd.DataFrame(labels, columns=["File name", TARGET_LABEL])
print("X shape:", X_df.shape)

if not VALIDATE:
    X_test = []
    with tqdm(total=len(images_test)) as pbar:
        for i in range(len(images_test)):
            i_features = []
            featureVector = extractor.execute(images_test[i], masks_test[i])    
            for key, value in featureVector.items():
                if 'diagnostics' not in key:        
                    i_features.append(float(value))
            X_test.append(i_features)
            pbar.update(1)
    
    # Average per-view features for every subject
    X_test = np.array(X_test)
    for j in np.unique(groups_test):
        j_inds = np.argwhere(groups_test == j)[:, 0]
        X_test[j_inds] = np.tile(np.mean(X_test[j_inds], 0), (len(j_inds), 1))

    X_df_test = pd.DataFrame(X_test, columns=feature_names)
    labels_df_test = pd.DataFrame(labels_test, columns=["File name", TARGET_LABEL])
    print("X_test shape:", X_df_test.shape)

## Classification 

In [ ]:
def aggregate_scores(groups, scores, operation = 'most_common'):
    # Aggregates predictions
    scores_final = []
    scores_grouped = []
    
    curr_group = groups[0]
    curr_scores = [scores[0]]
    for i in range(1, len(groups)):
        if groups[i] == curr_group:
            curr_scores.append(scores[i])
            if i == len(groups)-1:
                scores_grouped.append(curr_scores)
        else:
            scores_grouped.append(curr_scores)
            curr_group = groups[i]
            curr_scores = [scores[i]]
            if i == len(groups)-1:
                scores_grouped.append(curr_scores)
                
    assert len(scores_grouped) == len(np.unique(groups))
    if operation == 'most_common':
        scores_final = [np.argmax(np.bincount(np.array(group))) for group in scores_grouped]
    else:
        scores_final = [operation(group) for group in scores_grouped]
    scores_final = np.array(scores_final)
    
    assert len(scores_final) == len(np.unique(groups))
    return scores_final 

#### Random Guess

In [ ]:
# Fair coin flip
if VALIDATE:
    results = np.empty((K_FOLDS, 6)) 
else:
    accuracy = 0.0
    f1_macro = 0.0
    bal_acc = 0.0
    auroc = 0.0
    aupr = 0.0
    
np.random.seed(SEED)
B = 500

for b in range(B):
    kfold = StratifiedGroupKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(X_df, labels_df[TARGET_LABEL], groups)):  
        if fold > 0 and not VALIDATE:
            break
        if VALIDATE:
            y_val = labels_df[TARGET_LABEL].loc[val_ids] 
            groups_val = np.array(groups)[val_ids]

            y_val_prob = np.random.uniform(size=len(y_val))
            y_val_prob = aggregate_scores(groups_val, y_val_prob, operation = np.mean)
            y_val_pred = (y_val_prob > 0.5)*1
            y_val_true = aggregate_scores(groups_val, y_val.to_numpy(), operation = 'most_common') 

            accuracy = accuracy_score(y_val_true, y_val_pred)
            f1_macro = f1_score(y_val_true, y_val_pred, average='macro', zero_division=0)
            bal_acc = balanced_accuracy_score(y_val_true, y_val_pred)
            auroc = roc_auc_score(y_val_true, y_val_prob)
            precision, recall, thresholds = precision_recall_curve(y_val_true, y_val_prob)
            aupr = auc(recall, precision)
            brier_score = np.mean((y_val_prob - y_val_true)**2)
            
            results[fold, 0] += accuracy
            results[fold, 1] += f1_macro
            results[fold, 2] += bal_acc
            results[fold, 3] += auroc
            results[fold, 4] += aupr
            results[fold, 5] += brier_score
        else:
            y_test_prob = np.random.uniform(size=len(labels_df_test))
            y_test_prob = aggregate_scores(groups_test, y_test_prob, operation = np.mean)
            y_test_pred = (y_test_prob > 0.5).astype(int)
            y_test_true = aggregate_scores(groups_test, labels_df_test[TARGET_LABEL].to_numpy(), 
                                           operation = 'most_common')

            accuracy += accuracy_score(y_test_true, y_test_pred)
            f1_macro += f1_score(y_test_true, y_test_pred, average='macro', zero_division=0)
            bal_acc += balanced_accuracy_score(y_test_true, y_test_pred)
            auroc += roc_auc_score(y_test_true, y_test_prob)
            precision_, recall_, thresholds_ = precision_recall_curve(y_test_true, y_test_prob)
            aupr += auc(recall_, precision_)
        
metric_names = ['Accuracy', 'F1_macro', 'Balanced accuracy', 'AUROC', 'AUPR', 'Brier']
if VALIDATE:
    print(f'\n{K_FOLDS}-FOLD CROSS VALIDATION RESULTS: DUMMY')
    print()
    agg_results_df = pd.DataFrame([np.mean(results, axis=0), np.std(results, axis=0)], columns=metric_names, 
                                  index=['mean', 'std'])
    print(agg_results_df)      
else:
    results_df = pd.DataFrame([np.array([accuracy/B, f1_macro/B, bal_acc/B, auroc/B, aupr/B, brier/B])], 
                              columns=metric_names, index=['test set'])   
    print(results_df)

#### Random forest 

In [ ]:
# RF classifier
B = 10

if VALIDATE:
    results = np.empty((K_FOLDS, 11))
    FP_codes_folds = []
    FN_codes_folds = []
else:
    accuracy = np.zeros(B)
    f1_macro = np.zeros(B)
    bal_acc = np.zeros(B)
    auroc = np.zeros(B)
    aupr = np.zeros(B)
    fpr_at_75 = np.zeros(B)
    fpr_at_80 = np.zeros(B)
    fpr_at_90 = np.zeros(B)
    fpr_at_95 = np.zeros(B)
    fpr_at_99 = np.zeros(B)
    brier = np.zeros(B)

for b in range(B):
    kfold = StratifiedGroupKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED+b)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(X_df, labels_df[TARGET_LABEL], groups)):  
        if fold > 0 and not VALIDATE:
            break
        if VALIDATE:
            X_train = X_df.loc[train_ids]
            y_train = labels_df.loc[train_ids]        
            groups_train = np.array(groups)[train_ids]
            X_val = X_df.loc[val_ids]
            y_val = labels_df.loc[val_ids]
            groups_val = np.array(groups)[val_ids]
        else:
            X_train = X_df
            y_train = labels_df
            groups_train = np.array(groups)

        kfold_inner = StratifiedGroupKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

        pipe = Pipeline([
            ('selector1', VarianceThreshold(threshold=0.0001)),
            ('selector2', SelectKBest(f_classif)), 
            ('classifier', RandomForestClassifier(random_state=SEED+b, class_weight='balanced'))
                        ])   
        param_grid = {
            'classifier__max_depth': [5, 10, 15], 
            'classifier__n_estimators': [100, 200],
            'selector2__k': [20, 40, 60]
                     }

        search = GridSearchCV(pipe, param_grid, n_jobs=-1, scoring=make_scorer(f1_score, average='macro'), 
                              cv=kfold_inner.split(X_train, y_train[TARGET_LABEL], groups_train))

        search.fit(X_train, y_train[TARGET_LABEL].values.ravel()) 

        if VALIDATE:
            y_val_prob = search.predict_proba(X_val)[:,1]

            y_val_prob = aggregate_scores(groups_val, y_val_prob, operation = np.mean)
            y_val_pred = (y_val_prob > 0.5)*1

            y_val_true = aggregate_scores(groups_val, y_val[TARGET_LABEL].to_numpy(), operation = 'most_common')
            groups_val = aggregate_scores(groups_val, groups_val, operation = 'most_common')   

            accuracy = accuracy_score(y_val_true, y_val_pred)
            f1_macro = f1_score(y_val_true, y_val_pred, average='macro', zero_division=0)
            bal_acc = balanced_accuracy_score(y_val_true, y_val_pred)
            auroc = roc_auc_score(y_val_true, y_val_prob)
            precision, recall, thresholds = precision_recall_curve(y_val_true, y_val_prob)
            aupr = auc(recall, precision)

            fpr, fnr, _ = det_curve(y_val_true, y_val_prob)
            tpr = 1 - fnr
            fpr_at_k = {}
            ks = [0.75, 0.80, 0.90, 0.95, 0.99]
            for k in ks:
                ind = np.argmin(np.abs(tpr - k))
                fpr_at_k['FPR at ' + str(k)] = np.round(fpr[ind], 3)
            
            brier_score = np.mean((y_val_prob - y_val_true)**2)
            
            results[fold, 0] = accuracy
            results[fold, 1] = f1_macro
            results[fold, 2] = bal_acc
            results[fold, 3] = auroc
            results[fold, 4] = aupr
            results[fold, 5] = fpr_at_k['FPR at 0.75']
            results[fold, 6] = fpr_at_k['FPR at 0.8']
            results[fold, 7] = fpr_at_k['FPR at 0.9']
            results[fold, 8] = fpr_at_k['FPR at 0.95']
            results[fold, 9] = fpr_at_k['FPR at 0.99']
            results[fold, 10] = brier_score

            FP = np.intersect1d(np.argwhere(y_val_pred==1).flatten(), np.argwhere(y_val_true==0).flatten()) 
            FN = np.intersect1d(np.argwhere(y_val_pred==0).flatten(), np.argwhere(y_val_true==1).flatten())

            FP_codes_folds.append(groups_val[FP])
            FN_codes_folds.append(groups_val[FN])
        else:
            y_test_prob = search.predict_proba(X_df_test)[:,1]
            y_test_prob = aggregate_scores(np.array(groups_test), y_test_prob, operation = np.mean)
            y_test_pred = (y_test_prob > 0.5)*1

            y_test_true = aggregate_scores(np.array(groups_test), labels_df_test[TARGET_LABEL].to_numpy(), 
                                           operation = 'most_common')       

            accuracy[b] = accuracy_score(y_test_true, y_test_pred)
            f1_macro[b] = f1_score(y_test_true, y_test_pred, average='macro', zero_division=0)
            bal_acc[b] = balanced_accuracy_score(y_test_true, y_test_pred)
            auroc[b] = roc_auc_score(y_test_true, y_test_prob)
            precision, recall, thresholds = precision_recall_curve(y_test_true, y_test_prob)
            aupr[b] = auc(recall, precision)
            
            fpr, fnr, _ = det_curve(y_test_true, y_test_prob)
            tpr = 1 - fnr
            fpr_at_k = {}
            ks = [0.75, 0.80, 0.90, 0.95, 0.99]
            for k in ks:
                ind = np.argmin(np.abs(tpr - k))
                fpr_at_k['FPR at ' + str(k)] = np.round(fpr[ind], 3)
            
            fpr_at_75[b] = fpr_at_k['FPR at 0.75']
            fpr_at_80[b] = fpr_at_k['FPR at 0.8']
            fpr_at_90[b] = fpr_at_k['FPR at 0.9']
            fpr_at_95[b] = fpr_at_k['FPR at 0.95']
            fpr_at_99[b] = fpr_at_k['FPR at 0.99']
            
            brier[b] = np.mean((y_test_prob - y_test_true)**2)

metric_names = ['Accuracy', 'F1_macro', 'Balanced accuracy', 'AUROC', 'AUPR', 'FPR at 75', 'FPR at 80', 
                'FPR at 90', 'FPR at 95', 'FPR at 99', 'Brier score']
if VALIDATE:
    print(f'\n{K_FOLDS}-FOLD CROSS VALIDATION RESULTS: RANDOM FOREST')
    print()
    agg_results_df = pd.DataFrame([np.mean(results, axis=0), np.std(results, axis=0)], 
                                  columns=metric_names, index=["mean", "std"])
    print(agg_results_df)      
else:
    results_df = pd.DataFrame([np.array([accuracy.sum()/B, f1_macro.sum()/B, bal_acc.sum()/B, 
                                         auroc.sum()/B, aupr.sum()/B], fpr_at_75.sum()/B, fpr_at_80.sum()/B,
                                         fpr_at_90.sum()/B, fpr_at_95.sum()/B, fpr_at_99.sum()/B, 
                                         brier.sum()/B)], 
                              columns=metric_names, index=['test set'])
    print(str(np.round(accuracy.mean(), 3)) + '+/-' + str(np.round(accuracy.std(), 3)) + '; ' + 
          str(np.round(f1_macro.mean(), 3)) + '+/-' + str(np.round(f1_macro.std(), 3)) + '; ' +
          str(np.round(bal_acc.mean(), 3)) + '+/-' + str(np.round(bal_acc.std(), 3)) + '; ' + 
          str(np.round(auroc.mean(), 3)) + '+/-' + str(np.round(auroc.std(), 3)) + '; ' +
          str(np.round(aupr.mean(), 3)) + '+/-' + str(np.round(aupr.std(), 3)) + '; ' + 
          str(np.round(fpr_at_75.mean(), 3)) + '+/-' + str(np.round(fpr_at_75.std(), 3)) + '; ' + 
          str(np.round(fpr_at_80.mean(), 3)) + '+/-' + str(np.round(fpr_at_80.std(), 3)) + '; ' +
          str(np.round(fpr_at_90.mean(), 3)) + '+/-' + str(np.round(fpr_at_90.std(), 3)) + '; ' +
          str(np.round(fpr_at_95.mean(), 3)) + '+/-' + str(np.round(fpr_at_95.std(), 3)) + '; ' +
          str(np.round(fpr_at_99.mean(), 3)) + '+/-' + str(np.round(fpr_at_99.std(), 3)) + '; ' +
          str(np.round(brier.mean(), 3)) + '+/-' + str(np.round(brier.std(), 3)))